In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
import uuid
from collections import defaultdict
import numpy as np
import pandas as pd
from spacy.lang.en import English
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
import lalaes2 as mylib

In [2]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [3]:
df = pd.read_csv("input/train.csv", low_memory=False)
cols = ["score"]
df[cols] = df[cols].astype(np.int8)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   essay_id   17307 non-null  object
 1   full_text  17307 non-null  object
 2   score      17307 non-null  int8  
dtypes: int8(1), object(2)
memory usage: 287.4+ KB


In [4]:
pre = mylib.BasicPreprocessor()


def preprocess_text(col) -> Callable:
    def fn(row) -> str:
        return pre(row[col])
    
    return fn


df["clean_text"] = df.progress_apply(preprocess_text("full_text"), axis=1)
df.info()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17307/17307 [00:05<00:00, 3443.58it/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17307 entries, 0 to 17306
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   essay_id    17307 non-null  object
 1   full_text   17307 non-null  object
 2   score       17307 non-null  int8  
 3   clean_text  17307 non-null  object
dtypes: int8(1), object(3)
memory usage: 422.7+ KB


In [5]:
df.to_parquet(f"output/comp.parquet", index=False)
assert df.notna().all(axis=None)

In [6]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:00:05.693026
